In [31]:
import pip
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from MulticoreTSNE import MulticoreTSNE as TSNE
from sklearn import decomposition
from tabulate import tabulate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
song_app = pd.read_csv("SongApp.csv",header=-1,)

In [3]:
song_app.head()

,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,NaN,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,0.0,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,1.0,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2.0,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,3.0,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [4]:
# clean data
song_app = song_app.dropna()
song_app.__delitem__(0)
num_columns = song_app.columns.shape[0]
columns = list(range(num_columns - 1))
columns = ['labels'] + list(map(str,columns))
song_app.columns = columns
song_app.head()

,labels,0,1,2,3,4,5,6,7,8,...,80,81,82,83,84,85,86,87,88,89
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
5,2001,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,...,6.59753,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263


In [9]:
df = song_app.sample(50000)

In [13]:
# split training and test set
train, test = train_test_split(df, train_size=0.5, random_state=90)
y_train = train['labels']
X_train = train.drop('labels', axis=1)
y_test = test['labels']
X_test = test.drop('labels', axis=1)

In [16]:
# scale features
scaler = preprocessing.StandardScaler().fit(X_train)
Xtrain_scaled = scaler.transform(X_train)

In [27]:
#Random Forest Classifier feature importances
rf = RandomForestClassifier(n_estimators=8, n_jobs=6)
rf.fit(Xtrain_scaled, y_train)
headers = ["name", "score"]
values = sorted(zip(columns[1:], rf.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

  name       score
     0  0.015493
     1  0.0130376
     5  0.0123118
     2  0.0122589
    42  0.0121969
     6  0.0121238
    11  0.0120173
    25  0.0118718
    87  0.0118151
    10  0.011811
    58  0.0117704
    61  0.0117672
    16  0.011728
    82  0.0116838
    40  0.0116179
    72  0.0115576
    39  0.0115483
    73  0.0115461
    13  0.011537
    63  0.0114649
    65  0.011431
    76  0.0114092
    36  0.0113627
    47  0.0113594
    79  0.0113301
    49  0.0113193
    34  0.0113053
    27  0.0113015
    60  0.0112978
     8  0.0112729
    84  0.0112722
    78  0.0112586
    24  0.0112585
    45  0.0112525
    50  0.0112028
    69  0.0111849
    62  0.0111846
    19  0.01117
    70  0.0111408
    77  0.0111249
    38  0.0111185
    74  0.0110994
     7  0.0110911
     4  0.0110821
    75  0.0110812
    56  0.0110704
    67  0.0110653
    52  0.0110426
    37  0.0110084
    33  0.0109717
    28  0.0109635
    32  0.0109484
    85  0.0109034
    54  0.0108992
    53  0.010884

In [36]:
pca = decomposition.PCA(n_components=89)
pca.fit(X_train)
X = pca.transform(X_train)